## Imports, definitions and setup

In [ ]:
!git clone https://github.com/peiva-git/deep_learning_project.git
%cd deep_learning_project
!pip install -e .
!pip install --upgrade numpy

In [ ]:
import dlproject as dlp
import matplotlib.pyplot as plt
import os.path
import numpy as np

### Load the MNIST dataset

In [ ]:
# Uncomment one line to select the desired noise level

noise_level = 'high'
# noise_level = 'med'
# noise_level = 'low'

if noise_level == 'high':
  noise_value = 0.7
elif noise_level == 'med':
  noise_value = 0.4
elif noise_level == 'low':
  noise_value = 0.1

In [ ]:
dataset_builder = dlp.data.MNISTDatasetBuilder()
dataset_builder.preprocess_dataset_simple_ae(noise_value)
train_data, test_data = dataset_builder.train_x, dataset_builder.test_x
noisy_train_data, noisy_test_data = dataset_builder.noisy_train_data, dataset_builder.noisy_test_data

In [ ]:
number_of_previews = 10
plt.figure(figsize=(20, 2))
for i in range(1, number_of_previews + 1):
    ax = plt.subplot(1, number_of_previews, i)
    plt.imshow(noisy_train_data[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Instantiate the model

In [ ]:
autoencoder_mnist = dlp.models.SimpleAutoencoder(input_shape=(28, 28, 1))
autoencoder_mnist.model.compile(optimizer='adam', loss='binary_crossentropy')
model = autoencoder_mnist.model

## Train the model

### Testing the model

First, we train the model to reconstruct the image that's given as an input. The reconstructed images should be similar, but not exactly the same.
We also save the model for later use.

In [ ]:
model.fit(
    x=train_data,
    y=train_data,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data)
)

Display the results.

In [ ]:
decoded_imgs = model.predict(test_data)

number_of_previews = 10
plt.figure(figsize=(20, 4))
for i in range(1, number_of_previews + 1):
    # Display original
    ax = plt.subplot(2, number_of_previews, i)
    plt.imshow(test_data[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, number_of_previews, i + number_of_previews)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Denoise images

Secondly, we retrain the model to reconstruct the image from a noisy input.

In [ ]:
print(os.path.join(os.getcwd(), 'models', f'{model.name}_mnist_{noise_level}.keras'))

In [ ]:
from tensorflow.keras.saving import load_model

model_path = os.path.join(os.getcwd(), 'models',
                               f'{model.name}_mnist_{noise_level}_noise.keras')

if os.path.exists(model_path):
  model = load_model(model_path)
else:
  if not os.path.exists(os.path.join(os.getcwd(), 'models')):
    os.mkdir(os.path.join(os.getcwd(), 'models'))

  model.fit(
      x=noisy_train_data,
      y=train_data,
      epochs=100,
      batch_size=128,
      shuffle=True,
      validation_data=(noisy_test_data, test_data)
  )

model.save(os.path.join('models', f'{model.name}_mnist_{noise_level}_noise.keras'))

Let's take a look at the results. Top, the ground truth digits fed to the network, than the noisy version and finally the digits are reconstructed by the network. It seems to work pretty well.

In [ ]:
import random

def display_random_images(test_data, noisy_test_data, model, num_images=10):
    # Randomly select 10 indices from the test dataset
    random_indices = random.sample(range(len(test_data)), num_images)

    plt.figure(figsize=(15, 4))

    for i, idx in enumerate(random_indices):
        # Original clean image
        plt.subplot(3, num_images, i + 1)
        plt.imshow(test_data[idx].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Noisy image
        plt.subplot(3, num_images, num_images + i + 1)
        plt.imshow(noisy_test_data[idx].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Predicted output from the autoencoder
        predicted_output = model.predict(noisy_test_data[idx].reshape(1, 28, 28, 1))
        plt.subplot(3, num_images, 2 * num_images + i + 1)
        plt.imshow(predicted_output[0].reshape(28, 28), cmap='gray')
        plt.axis('off')

    plt.show()

display_random_images(test_data, noisy_test_data, model)

Compute predictions for the entire noisy MNIST dataset.

Compute PSNR and SSIM for model evaluation.

In [ ]:
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity as ssim

def compute_mean_psnr(test_data, reconstructed_test_data):
    # Compute the mean PSNR (Peak Signal-to-Noise Ratio) for a set of image pairs

    if len(test_data) != len(noisy_test_data):
        raise ValueError("Input lists must have the same length")

    psnr_values = []

    for original, noisy in zip(test_data, noisy_test_data):
        psnr = peak_signal_noise_ratio(original, noisy)
        psnr_values.append(psnr)

    mean_psnr = np.mean(psnr_values)
    return mean_psnr

def compute_mean_ssim(test_data, noisy_test_data):
    # Compute the mean SSIM (Structural Similarity Index) for a set of image pairs

    if len(test_data) != len(noisy_test_data):
        raise ValueError("Input lists must have the same length")

    ssim_values = []

    for original, noisy in zip(test_data, noisy_test_data):
        ssim_score = ssim(original, noisy, multichannel=True)
        ssim_values.append(ssim_score)

    mean_ssim = np.mean(ssim_values)
    return mean_ssim


In [ ]:
reconstructed_images = model.predict(noisy_test_data)
print(compute_mean_psnr(test_data, reconstructed_images))
print(compute_mean_ssim(test_data, reconstructed_images))

# Model with CIFAR10 dataset

## Load the CIFAR10 dataset

In [ ]:
dataset_builder = dlp.data.CIFAR10DatasetBuilder()
dataset_builder.preprocess_dataset_simple_ae(0.1)
train_data, test_data = dataset_builder.train_x, dataset_builder.test_x
noisy_train_data, noisy_test_data = dataset_builder.noisy_train_data, dataset_builder.noisy_test_data

In [ ]:
import random

def display_random_images(train_data, noisy_train_data, num_images=8):
    fig, axes = plt.subplots(2, num_images, figsize=(16, 4))

    for i in range(num_images):
        index = random.randint(0, train_data.shape[0] - 1)

        # Display a random image from the clean dataset
        axes[0, i].imshow(train_data[index])
        axes[0, i].set_title("Clean")

        # Display the corresponding image from the noisy dataset
        axes[1, i].imshow(noisy_train_data[index])
        axes[1, i].set_title("Noisy")

        # Remove axis labels
        axes[0, i].axis('off')
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.show()

# Display 8 random images from the datasets
display_random_images(train_data, noisy_train_data, num_images=8)

## Istantiate the model for CIFAR10 dataset

In [ ]:
autoencoder_cifar = dlp.models.SimpleAutoencoder(input_shape=(32, 32, 3))
autoencoder_cifar.model.compile(optimizer='adam', loss='binary_crossentropy')
model = autoencoder_cifar.model

# Train the CIFAR10 model



In [ ]:
from tensorflow.keras.saving import load_model

model_path = os.path.join(os.getcwd(), 'models', f'{model.name}_cifar_low_noise.keras')

if os.path.exists(model_path):
  model = load_model(model_path)
else:
  if not os.path.exists(os.path.join(os.getcwd(), 'models')):
    os.mkdir(os.path.join(os.getcwd(), 'models'))

  model.fit(
      x=noisy_train_data,
      y=train_data,
      epochs=100,
      batch_size=128,
      shuffle=True,
      validation_data=(noisy_test_data, test_data)
  )
  autoencoder_cifar.model.save(os.path.join(os.getcwd(), 'models', f'{model.name}_cifar.keras'))

Display some results.

In [ ]:
import random
import matplotlib.pyplot as plt

def display_random_images(test_data, noisy_test_data, autoencoder, num_images=10):
    # Randomly select 10 indices from the test dataset
    random_indices = random.sample(range(len(test_data)), num_images)

    plt.figure(figsize=(15, 4))

    for i, idx in enumerate(random_indices):
        # Original clean image
        plt.subplot(3, num_images, i + 1)
        plt.imshow(test_data[idx])
        plt.axis('off')

        # Noisy image
        plt.subplot(3, num_images, num_images + i + 1)
        plt.imshow(noisy_test_data[idx])
        plt.axis('off')

        # Predicted output from the autoencoder
        predicted_output = model.predict(noisy_test_data[idx].reshape(1, 32, 32, 3))
        plt.subplot(3, num_images, 2 * num_images + i + 1)
        plt.imshow(predicted_output[0])
        plt.axis('off')

    plt.show()

display_random_images(test_data, noisy_test_data, model, num_images=10)

Compute and display the PSNR and SSIM metrics.

In [ ]:
reconstructed_images = model.predict(noisy_test_data)
print(compute_mean_psnr(test_data, reconstructed_images))
print(compute_mean_ssim(test_data, reconstructed_images))